# SMRT-Tag
#### 08/03/2022
This notebook demonstrates how to process SMRT-Tag data, and perform the set of analyses described in our associated manuscript. 

In [ ]:
## Imports
import os

In [ ]:
## Get path of notebook 
NOTEBOOK_PATH = os.getcwd()
## Set working directory paths
BASE_DIR=os.path.join(os.path.dirname(NOTEBOOK_PATH))
TOP_DIR=os.path.join(BASE_DIR,'smrt_tag')
## Specify the top-level directory 
TOP_DIR='/Users/snanda/storage/lab/projects/SMRT-Tag/smrt_tag/'

In [ ]:
## Create directory structure 
directories=[
 'ref',
 'ref/GRCh37/',
 'ref/GRCh38/',
 'raw/HG/',
 'preprocess/align',
 'analyses/HG/demultiplex_genotype/calling_regions',
 'analyses/HG/demultiplex_genotype/calling_regions/unique',
 'analyses/HG/demultiplex_genotype/align/',
 'analyses/HG/demultiplex_genotype/align/mosdepth/',
 'analyses/HG/demultiplex_genotype/plp/',
 'analyses/HG/demultiplex_genotype/plp_cmp/',
 'analyses/HG/variant_calling/',
 'analyses/HG/variant_calling/align',
 'analyses/HG/variant_calling/downsample',
 'analyses/HG/variant_calling/deepvariant'
]

for directory in directories:
    os.makedirs("{}/{}".format(TOP_DIR,directory),exist_ok=True)

### 1.) Download reference datasets

In [ ]:
%%bash
#!/usr/bin/env bash

## hg38 reference genom
    
wget ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/genome-stratifications/v3.0/v3.0-stratifications-GRCh37.tar.gz \
    -O "$TOP_DIR/ref/v3.0-stratifications-GRCh37.tar.gz" && \
    tar -xf "$TOP_DIR/ref/v3.0-stratifications-GRCh37.tar.gz" -C "$TOP_DIR/ref/"

## GRCh37 reference genome
wget ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/references/GRCh37/hs37d5.fa.gz \
    -O "$TOP_DIR/ref/GRCh37/hs37d5.fa.gz" && \
    gunzip -c "$TOP_DIR/ref/GRCh37/hs37d5.fa.gz" > "$TOP_DIR/ref/GRCh37/hs37d5.fa"

## GRCh38 reference genome:
wget ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/references/GRCh38/GCA_000001405.15_GRCh38_no_alt_analysis_set.fasta.gz \
    -O "$TOP_DIR/ref/GRCh38/hg38.fa.gz" && \
        gunzip -c "$TOP_DIR/ref/GRCh38/hg38.fa.gz" > "$TOP_DIR/ref/GRCh38/hg38.fa"

## HG002 
wget ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/AshkenazimTrio/HG002_NA24385_son/NISTv4.2.1/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark.vcf.gz \
    -O "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark.vcf.gz"
    
wget ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/AshkenazimTrio/HG002_NA24385_son/NISTv4.2.1/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed \
    -O "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed" 
    
## HG003
wget ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/AshkenazimTrio/HG003_NA24149_father/NISTv4.2.1/GRCh37/HG003_GRCh37_1_22_v4.2.1_benchmark.vcf.gz \
    -O "$TOP_DIR/ref/GRCh37/HG003_GRCh37_1_22_v4.2.1_benchmark.vcf.gz"
    
wget ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/AshkenazimTrio/HG003_NA24149_father/NISTv4.2.1/GRCh37/HG003_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed \
    -O "$TOP_DIR/ref/GRCh37/HG003_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed "
        
## HG004
wget ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/AshkenazimTrio/HG004_NA24143_mother/NISTv4.2.1/GRCh37/HG004_GRCh37_1_22_v4.2.1_benchmark.vcf.gz \
    -O "$TOP_DIR/ref/GRCh37/HG004_GRCh37_1_22_v4.2.1_benchmark.vcf.gz"
    
wget ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/AshkenazimTrio/HG004_NA24143_mother/NISTv4.2.1/GRCh37/HG004_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed \
    -O "$TOP_DIR/ref/GRCh37/HG004_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed"
        
## PacBio Sequel II 11kb reads HG002
wget ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/HG002_NA24385_son/PacBio_SequelII_CCS_11kb/HG002.SequelII.pbmm2.hs37d5.whatshap.haplotag.RTG.10x.trio.bam \
    -O "$TOP_DIR/ref/HG002-ctrl.SequelII.pbmm2.hs37d5.whatshap.haplotag.RTG.10x.trio.bam"

## HG002 CpG bismark processed data
wget https://ont-open-data.s3.amazonaws.com/gm24385_mod_2021.09/bisulphite/cpg/CpG.gz.bismark.zero.cov.gz \
     -O "$TOP_DIR/ref/CpG.gz.bismark.zero.cov.gz"
     


### 2.) Download SRA and GEO datasets

#### 3a.) Merge demultiplexed BAM files together by condition

In [ ]:
%%bash
#!/usr/bin/env bash
# merge_HG_ccs_data.sh: bash script to merge HHG{002,003,004} SMRT-Tag ccs bam files by condition
# Usage: ./merge_HG_ccs_data.sh
set -eo pipefail

## HG002
pbmerge -j 15 \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A16.demux2.HG002_PR28--HG002_PR28.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A16.demux2.HG002_PR29--HG002_PR29.bam" \
    -o "${TOP_DIR}/raw/HG/SMRT-Tag.A16.demux2.HG002.bam"

## HG003
pbmerge -j 15 \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A16.demux2.HG003_PR33--HG003_PR33.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A16.demux2.HG003_PR34--HG003_PR34.bam" \
    -o "${TOP_DIR}/raw/HG/SMRT-Tag.A16.demux2.HG003.bam"

## HG004
pbmerge -j 15 \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A16.demux2.HG004_PR30--HG004_PR30.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A16.demux2.HG004_PR31--HG004_PR31.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A16.demux2.HG004_PR32--HG004_PR32.bam" \
    -o "${TOP_DIR}/raw/HG/SMRT-Tag.A16.demux2.HG004.bam"
    
## All HG002 bams
pbmerge -j 15 \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A08.demux2.HG002_PR27--HG002_PR27.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A08.demux2.HG002_PR28--HG002_PR28.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A09.demux2.HG002_PR12--HG002_PR12.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A09.demux2.HG002_PR27--HG002_PR27.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A09.demux2.HG002_PR29--HG002_PR29.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A09.demux2.HG002_PR30--HG002_PR30.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A09.demux2.HG002_PR31--HG002_PR31.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A09.demux2.HG002_PR32--HG002_PR32.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A16.demux2.HG002_PR28--HG002_PR28.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A16.demux2.HG002_PR29--HG002_PR29.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A17.demux2.HG002_PR27--HG002_PR27.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A17.demux2.HG002_PR28--HG002_PR28.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A17.demux2.HG002_PR29--HG002_PR29.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A17.demux2.HG002_PR30--HG002_PR30.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A17.demux2.HG002_PR31--HG002_PR31.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A17.demux2.HG002_PR32--HG002_PR32.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A17.demux2.HG002_PR33--HG002_PR33.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A17.demux2.HG002_PR34--HG002_PR34.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.long.demux2.HG002_PR35--HG002_PR35.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.long.demux2.HG002_PR36--HG002_PR36.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.long.demux2.HG002_PR37--HG002_PR37.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.long.demux2.HG002_PR38--HG002_PR38.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.long.demux2.HG002_PR39--HG002_PR39.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.long.demux2.HG002_PR40--HG002_PR40.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.long.demux2.HG002_PR41--HG002_PR41.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.long.demux2.HG002_PR42--HG002_PR42.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.short.demux2.HG002_PR35--HG002_PR35.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.short.demux2.HG002_PR36--HG002_PR36.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.short.demux2.HG002_PR37--HG002_PR37.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.short.demux2.HG002_PR38--HG002_PR38.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.short.demux2.HG002_PR39--HG002_PR39.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.short.demux2.HG002_PR40--HG002_PR40.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.short.demux2.HG002_PR41--HG002_PR41.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A18.short.demux2.HG002_PR42--HG002_PR42.bam" \
    -o "${TOP_DIR}/raw/HG/SMRT-Tag.demux2.HG002.bam"



#### 3b.) Run primrose on HG002 bam data

In [ ]:
%%bash
#!/usr/bin/env bash
# run_primrose_HG002_ccs_data.sh: bash script to run primrose on merged HG002 SMRT-Tag ccs data
# Usage: ./run_primrose_HG002_ccs_data.sh
set -eo pipefail
primrose \
    -j 30 \
    --log-level INFO \
    --keep-kinetics \
    "${TOP_DIR}/raw/HG/SMRT-Tag.demux2.HG002.bam" \
    "${TOP_DIR}/raw/HG/SMRT-Tag.demux2.HG002.primrose.bam"


#### 3c.) Align all files to the reference genome

In [ ]:
%%bash
#!/usr/bin/env bash
# align_HG_ccs_data.sh: bash script to align all HG{002,003,004} SMRT-Tag ccs data
# Usage: ./align_HG_ccs_data.sh

for ccs_file in $(ls "${TOP_DIR}/raw/HG/*HG*bam");
do;
    file=$(basename $ccs_file)
    outfile="${file/--*/.bam}"
    
    pbmm2 align \
    --preset CCS \
    --sort -j 30 \
    --bam-index BAI \
    --log-level DEBUG \
    $ccs_file \
    "$TOP_DIR/ref/GRCh38/hg38.fa" \
    "$TOP_DIR/preprocess/align/${outfile/.bam/.align.sorted.bam}"
    
done

# Secondary Analyses 

#### 4a.) Extract CpG information for 5mC comparision with bisulfite data

In [ ]:
%%bash
#!/usr/bin/env bash
# 05_cpg2pickle.sh: bash script to run 05_cpg2pickle.py, and extract primrose 5mC predictions per molecule from reads
# Usage: ./05_cpg2pickle.sh *primrose aligned BAM*
#     --project: Name of project being analyzed
#     --output-dir: Directory to save results
#
## Outputs:
#     output-dir/project_cpg_data.pickle: dictionary containing per-molecule 5mC methylation probability from all BAMs

$BASE_DIR/samosa_tag/05_cpg2pickle.py \
    "$TOP_DIR/preprocess/align/SMRT-Tag.demux2.HG002.primrose.align.sorted.bam" \
    --project HG002 \
    --output-dir "$TOP_DIR/analyses/HG/"
    

### 5) Genotype demultiplexing experiment

#### 5a.) Determine private variants for demultiplexing

In [ ]:
%%bash
#! /usr/bin/env bash
# demux_process_truth_sets.sh: Process reference regions and variants from HG{002,003,004} to produce shared calling regions and private variants
# Usage: ./demux_process_truth_sets.sh 
#
## Inputs:
#     TOPDIR: $TOP_DIR
#     HG002_VCF: HG002 GRCh37 benchmark variant calls
#     HG003_VCF: HG003 GRCh37 benchmark variant calls
#     HG004_VCF: HG004 GRCh37 benchmark variant calls
#     HG002_BED: HG002 GRCh37 verified variant calling regions
#     HG003_BED: HG003 GRCh37 verified variant calling regions
#     HG004_BED: HG004 GRCh37 verified variant calling regions
#
## Outputs:
#     Outputs are written to ${TOPDIR}/analyses/HG/demultiplex_genotype/calling_regions/:
#         HG002-4.calling_regions.bed.gz: Shared set of GRCh37 verified calling regions
#
#     Outputs are written to ${TOPDIR}/analyses/HG/demultiplex_genotype/calling_regions/unique/
#         unique.HG002_GRCh37_1_22_v4.2.1_benchmark.vcf.gz: Unique variants for HG002
#         unique.HG003_GRCh37_1_22_v4.2.1_benchmark.vcf.gz: Unique variants for HG003
#         unique.HG004_GRCh37_1_22_v4.2.1_benchmark.vcf.gz: Unique variants for HG004


${BASE_DIR}/demultiplex_genotype/demux_process_truth_sets.sh \
    $TOP_DIR \
    "$TOP_DIR/ref/HG002_GRCh37_1_22_v4.2.1_benchmark.vcf.gz" \
    "$TOP_DIR/ref/HG003_GRCh37_1_22_v4.2.1_benchmark.vcf.gz" \
    "$TOP_DIR/ref/HG004_GRCh37_1_22_v4.2.1_benchmark.vcf.gz" \
    "$TOP_DIR/ref/HG002_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed" \
    "$TOP_DIR/ref/HG003_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed" \
    "$TOP_DIR/ref/HG004_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed"


#### 5b.) Align BAMs to hs37d5

In [ ]:
%%bash
#! /usr/bin/env bash
# pbmm2_align.sh: Align PacBio unaligned BAM files to a reference genome and run mosdepth to calculate depth stats.
# Usage: ./pbmm2_align.sh 
#
## Inputs:
#     TOPDIR: $TOP_DIR
#     BAM: PacBio BAM file to be aligned
#     OUTPREFIX: Prefix to append to output files
#     OUTDIR: Directory to write output
#     FASTA: Reference genome FASTA file
#     STRAT_BED: Region file for depth calculations - GRCh37_notinalllowmapandsegdupregions.bed.gz
#
## Outputs:
#     Outputs are written to $OUTDIR
#         $OUTDIR/$OUTPREFIX.align.sorted.bam: Aligned BAM file
#         $OUTDIR/mosdepth/$OUTPREFIX.mosdepth* : mosdepth files including (per-base, regions, thresholds.bed etc.)


$BASE_DIR/variant_calling/pbmm2_align.sh \
    $TOP_DIR \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A16.demux2.HG002.bam" \
    HG002
    "${TOP_DIR}/analyses/HG/demultiplex_genotype/align/"
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa" \
    "${TOP_DIR}/ref/GRCh37/union/GRCh37_notinalllowmapandsegdupregions.bed.gz"
    
$BASE_DIR/variant_calling/pbmm2_align.sh \
    $TOP_DIR \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A16.demux2.HG003.bam" \
    HG003
    "${TOP_DIR}/analyses/HG/demultiplex_genotype/align/"
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa" \
    "${TOP_DIR}/ref/GRCh37/union/GRCh37_notinalllowmapandsegdupregions.bed.gz"
    
$BASE_DIR/variant_calling/pbmm2_align.sh \
    $TOP_DIR \
    "${TOP_DIR}/raw/HG/SMRT-Tag.A16.demux2.HG004.bam" \
    HG004
    "${TOP_DIR}/analyses/HG/demultiplex_genotype/align/"
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa" \
    "${TOP_DIR}/ref/GRCh37/union/GRCh37_notinalllowmapandsegdupregions.bed.gz"
    

#### 5c.) Naive variant calling in GIAB benchmark regions

In [ ]:
%%bash
#! /usr/bin/env bash
# demux_plp2vcf.sh: Naively calls variants using samtools mpileup in the regions provided
# Usage: ./demux_plp2vcf.sh 
#
## Inputs:
#     TOPDIR: $TOP_DIR
#     BAM: BAM-file to call variants
#     FASTA: GRCh37, hs37d5, reference genome fasta
#     CALLING_REGIONS: BED file containing shared set of GRCh37 verified calling regions produced by demux_process_truth_sets.sh
#
## Outputs:
#     Outputs are written to ${TOPDIR}/analyses/HG/demultiplex_genotype/plp/:
#         BAMfile.q15.d2.vcf.gz: Naive VCF containing SNVs called in the provided regions with at least 2 reads and a minimum MAPQ of 15

$BASE_DIR/demultiplex_genotype/demux_plp2vcf.sh \
    $TOP_DIR \
    "${TOP_DIR}/analyses/HG/demultiplex_genotype/align/HG002.align.sorted.bam" \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa" \
    "${TOPDIR}/analyses/HG//calling_regions/HG002-4.calling_regions.bed.gz"
    
$BASE_DIR/demultiplex_genotype/demux_plp2vcf.sh \
    $TOP_DIR \
    "${TOP_DIR}/analyses/HG/demultiplex_genotype/align/HG003.align.sorted.bam" \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa" \
    "${TOPDIR}/analyses/HG/demultiplex_genotype/calling_regions/HG002-4.calling_regions.bed.gz"
    
$BASE_DIR/demultiplex_genotype/demux_plp2vcf.sh \
    $TOP_DIR \
    "${TOP_DIR}/analyses/HG/demultiplex_genotype/align/HG004.align.sorted.bam" \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa" \
    "${TOPDIR}/analyses/HG/demultiplex_genotype/calling_regions/HG002-4.calling_regions.bed.gz"


#### 5d.) Intersect called variants with private variants to determine degree of shared variants 

In [ ]:
%%bash
#! /usr/bin/env bash
# demux_compare.sh: Intersect called variants with private variants to determine degree of shared variants 
# Usage: ./demux_compare.sh 
#
## Inputs:
#     TOPDIR: $TOP_DIR
#     BAM: BAM-file to call variants
#     REGIONS: Region file for calculations - GRCh37_notinalllowmapandsegdupregions.bed.gz
#
## Outputs:
#     Outputs are written to ${TOPDIR}/analyses/HG/demultiplex_genotype/plp_cmp/:
#         depth2_bed/HG{002,003,004}.d2.GRCh37_notinalldifficultregions.bed.gz: BED file indicating regions where at least 2 reads are present
#         HG{002,003,004}.q15.d2_vs_HG{002,003,004}_unique.vcf.gz: VCF file containing variants present in a sequenced sample and private benchmark variants.
#         HG{002,003,004}.q15.d2_vs_HG{002,003,004}_unique.stats: Stats for above VCF file
#         HG{002,003,004}.d2_vs_HG{002,003,004}.base.vcf.gz: VCF file containing shared variants in experimental regions (at least 2 reads are present)
#         HG{002,003,004}.d2_vs_HG{002,003,004}.base.stats: Stats for above VCF file

$BASE_DIR/demultiplex_genotype/demux_compare.sh \
    $TOP_DIR \
    "$TOP_DIR/ref/GRCh37/union/GRCh37_notinalldifficultregions.bed.gz"
    


### 6.) Variant Calling

#### 6a.) Aligning HG002 reads to the genome 

In [ ]:
%%bash
#! /usr/bin/env bash
# pbmm2_align.sh: Align PacBio unaligned BAM files to a reference genome and run mosdepth to calculate depth stats.
# Usage: ./pbmm2_align.sh 
#
## Inputs:
#     TOPDIR: $TOP_DIR
#     BAM: PacBio BAM file to be aligned
#     OUTPREFIX: Prefix to append to output files
#     OUTDIR: Directory to write output
#     FASTA: Reference genome FASTA file
#     REGIONS: Region file for depth calculations - GRCh37_notinalllowmapandsegdupregions.bed.gz
#
## Outputs:
#     Outputs are written to $OUTDIR
#         $OUTDIR/$OUTPREFIX.align.sorted.bam: Aligned BAM file
#         $OUTDIR/mosdepth/$OUTPREFIX.mosdepth* : mosdepth files including (per-base, regions, thresholds.bed etc.)


$BASE_DIR/variant_calling/pbmm2_align.sh \
    $TOP_DIR \
    "${TOP_DIR}/raw/HG/SMRT-Tag.demux2.HG002.bam" \
    HG002
    "${TOP_DIR}/analyses/HG/variant_calling/align/"
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa" \
    "${TOP_DIR}/ref/GRCh37/union/GRCh37_notinalllowmapandsegdupregions.bed.gz"
    

#### 6b.) Subsampling aligned BAM files 

In [ ]:
%%bash
#! /usr/bin/env bash
# downsample_bam.sh: Downsample aligned BAM files produced by pbmm2_align.sh
# Usage: ./downsample_bam.sh 
#
## Inputs:
#     TOPDIR: $TOP_DIR
#     BAM: PacBio BAM file to be aligned
#     DEPTH: Depth of downsampling
#     FRAC: Fraction provided to samtools for downsampling
#     REGIONS: Region file for depth calculations - GRCh37_notinalllowmapandsegdupregions.bed.gz
#
## Outputs:
#     Outputs are written to ${TOPDIR}/analyses/HG/variant_calling/downsample/${PREFIX}/${DEPTH}X/seed${SEED}
#         $OUTDIR/${PREFIX}.${DEPTH}X.seed${SEED}.bam: Downsampled aligned BAM file
#         $OUTDIR/mosdepth/$OUTPREFIX.mosdepth* : mosdepth files including (per-base, regions, thresholds.bed etc.)


$BASE_DIR/variant_calling/downsample_bam.sh \
    $TOP_DIR \
    "${TOP_DIR}/analyses/HG/variant_calling/align/HG002.align.sorted.bam" \
    10 \
    0.895 \
    "${TOP_DIR}/ref/GRCh37/union/GRCh37_notinalllowmapandsegdupregions.bed.gz"
    
$BASE_DIR/variant_calling/downsample_bam.sh \
    $TOP_DIR \
    "${TOP_DIR}/analyses/HG/variant_calling/align/HG002.align.sorted.bam" \
    5 \
    0.44 \
    "${TOP_DIR}/ref/GRCh37/union/GRCh37_notinalllowmapandsegdupregions.bed.gz"
    
    
$BASE_DIR/variant_calling/downsample_bam.sh \
    $TOP_DIR \
    "${TOP_DIR}/analyses/HG/variant_calling/align/HG002.align.sorted.bam" \
    3 \
    0.268 \
    "${TOP_DIR}/ref/GRCh37/union/GRCh37_notinalllowmapandsegdupregions.bed.gz"



## Subsample control data as well
$BASE_DIR/variant_calling/downsample_bam.sh \
    $TOP_DIR \
    "$TOP_DIR/ref/HG002-ctrl.SequelII.pbmm2.hs37d5.whatshap.haplotag.RTG.10x.trio.bam" \
    15 \
    1.0 \
    "${TOP_DIR}/ref/GRCh37/union/GRCh37_notinalllowmapandsegdupregions.bed.gz"

$BASE_DIR/variant_calling/downsample_bam.sh \
    $TOP_DIR \
    "$TOP_DIR/ref/HG002-ctrl.SequelII.pbmm2.hs37d5.whatshap.haplotag.RTG.10x.trio.bam" \
    10 \
    0.66 \
    "${TOP_DIR}/ref/GRCh37/union/GRCh37_notinalllowmapandsegdupregions.bed.gz"

$BASE_DIR/variant_calling/downsample_bam.sh \
    $TOP_DIR \
    "$TOP_DIR/ref/HG002-ctrl.SequelII.pbmm2.hs37d5.whatshap.haplotag.RTG.10x.trio.bam" \
    5 \
    0.33 \
    "${TOP_DIR}/ref/GRCh37/union/GRCh37_notinalllowmapandsegdupregions.bed.gz"

$BASE_DIR/variant_calling/downsample_bam.sh \
    $TOP_DIR \
    "$TOP_DIR/ref/HG002-ctrl.SequelII.pbmm2.hs37d5.whatshap.haplotag.RTG.10x.trio.bam" \
    3 \
    0.20 \
    "${TOP_DIR}/ref/GRCh37/union/GRCh37_notinalllowmapandsegdupregions.bed.gz"




#### 6c.) Running deepvariant

In [ ]:
%%bash
#! /usr/bin/env bash
# deepvariant.sh: Call variants from PacBio HiFi reads using deepvariant
# Usage: ./deepvariant.sh 
#
## Inputs:
#     TOPDIR: $TOP_DIR
#     BAM: PacBio BAM file to be 
#     OUTPREFIX: Prefix to append to output files
#     FASTA: Genome FASTA file used for alignment
#
## Outputs:
#     Outputs are written to ${TOPDIR}/analyses/HG/variant_calling/deepvariant/$OUTPREFIX
#          $OUTDIR/$OUTPREFIX.deepvariant.vcf.gz: Deepvariant VCF


$BASE_DIR/variant_calling/deepvariant.sh \
    $TOP_DIR \
    "${TOPDIR}/analyses/HG/variant_calling/downsample/HG002/10X/seed0/HG002.10X.seed0.bam" \
    HG002.10X.seed0 \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa"
    
    
    
$BASE_DIR/variant_calling/deepvariant.sh \
    $TOP_DIR \
    "${TOPDIR}/analyses/HG/variant_calling/downsample/HG002/5X/seed0/HG002.5X.seed0.bam" \
    HG002.5X.seed0 \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa"
    
    
$BASE_DIR/variant_calling/deepvariant.sh \
    $TOP_DIR \
    "${TOPDIR}/analyses/HG/variant_calling/downsample/HG002/3X/seed0/HG002.3X.seed0.bam" \
    HG002.3X.seed0 \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa"
    
    
    
$BASE_DIR/variant_calling/deepvariant.sh \
    $TOP_DIR \
    "${TOPDIR}/analyses/HG/variant_calling/downsample/HG002-ctrl/15X/seed0/HG002-ctrl.15X.seed0.bam" \
    HG002-ctrl.15X.seed0 \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa"
    
    
$BASE_DIR/variant_calling/deepvariant.sh \
    $TOP_DIR \
    "${TOPDIR}/analyses/HG/variant_calling/downsample/HG002-ctrl/10X/seed0/HG002-ctrl.10X.seed0.bam" \
    HG002-ctrl.10X.seed0 \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa"
    
    
$BASE_DIR/variant_calling/deepvariant.sh \
    $TOP_DIR \
    "${TOPDIR}/analyses/HG/variant_calling/downsample/HG002-ctrl/5X/seed0/HG002-ctrl.5X.seed0.bam" \
    HG002-ctrl.5X.seed0 \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa"
    

$BASE_DIR/variant_calling/deepvariant.sh \
    $TOP_DIR \
    "${TOPDIR}/analyses/HG/variant_calling/downsample/HG002-ctrl/3X/seed0/HG002-ctrl.3X.seed0.bam" \
    HG002-ctrl.3X.seed0 \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa"
    
    


#### 6d.) Evaluate deepvariant variant calls with hap.py

In [ ]:
%%bash
#! /usr/bin/env bash
# happy.sh: Benchmark vcf against truth set using hap.py
# Usage: ./happy.sh 
#
## Inputs:
#     TOPDIR: $TOP_DIR
#     VCF: Deepvariant VCF file
#     TRUTH_VCF: Truth VCF file from GIAB
#     TRUTH_BED: Truth BED file from GIAB
#     FASTA: Genome FASTA file used for alignment
#     STRAT: Genome stratifications for GRCh37 from GIAB - v3.0-GRCh37-v4.2.1-stratifications.tsv
#
## Outputs:
#     Outputs are written to ${TOPDIR}/analyses/HG/variant_calling/deepvariant/$OUTPREFIX:
#         $OUTDIR/$OUTPREFIX.deepvariant.happy: hap.py output file benchmarking variant calling


$BASE_DIR/variant_calling/happy.sh \
    $TOP_DIR \
    ${TOPDIR}/analyses/HG/variant_calling/deepvariant/HG002.10X.seed0/HG002.10X.seed0.deepvariant.vcf.gz \
    "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark.vcf.gz" \
    "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed" \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa" \
    "${TOP_DIR}/ref/GRCh37/v3.0-GRCh37-v4.2.1-stratifications.tsv"

$BASE_DIR/variant_calling/happy.sh \
    $TOP_DIR \
    ${TOPDIR}/analyses/HG/variant_calling/deepvariant/HG002.5X.seed0/HG002.5X.seed0.deepvariant.vcf.gz \
    "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark.vcf.gz" \
    "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed" \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa" \
    "${TOP_DIR}/ref/GRCh37/v3.0-GRCh37-v4.2.1-stratifications.tsv"

$BASE_DIR/variant_calling/happy.sh \
    $TOP_DIR \
    ${TOPDIR}/analyses/HG/variant_calling/deepvariant/HG002.3X.seed0/HG002.3X.seed0.deepvariant.vcf.gz \
    "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark.vcf.gz" \
    "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed" \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa" \
    "${TOP_DIR}/ref/GRCh37/v3.0-GRCh37-v4.2.1-stratifications.tsv"


$BASE_DIR/variant_calling/happy.sh \
    $TOP_DIR \
    ${TOPDIR}/analyses/HG/variant_calling/deepvariant/HG002-ctrl.15X.seed0/HG002-ctrl.15X.seed0.deepvariant.vcf.gz \
    "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark.vcf.gz" \
    "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed" \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa" \
    "${TOP_DIR}/ref/GRCh37/v3.0-GRCh37-v4.2.1-stratifications.tsv"

$BASE_DIR/variant_calling/happy.sh \
    $TOP_DIR \
    ${TOPDIR}/analyses/HG/variant_calling/deepvariant/HG002-ctrl.10X.seed0/HG002-ctrl.10X.seed0.deepvariant.vcf.gz \
    "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark.vcf.gz" \
    "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed" \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa" \
    "${TOP_DIR}/ref/GRCh37/v3.0-GRCh37-v4.2.1-stratifications.tsv"


$BASE_DIR/variant_calling/happy.sh \
    $TOP_DIR \
    ${TOPDIR}/analyses/HG/variant_calling/deepvariant/HG002-ctrl.5X.seed0/HG002-ctrl.5X.seed0.deepvariant.vcf.gz \
    "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark.vcf.gz" \
    "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed" \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa" \
    "${TOP_DIR}/ref/GRCh37/v3.0-GRCh37-v4.2.1-stratifications.tsv"


$BASE_DIR/variant_calling/happy.sh \
    $TOP_DIR \
    ${TOPDIR}/analyses/HG/variant_calling/deepvariant/HG002-ctrl.3X.seed0/HG002-ctrl.3X.seed0.deepvariant.vcf.gz \
    "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark.vcf.gz" \
    "$TOP_DIR/ref/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed" \
    "${TOP_DIR}/ref/GRCh37/hs37d5.fa" \
    "${TOP_DIR}/ref/GRCh37/v3.0-GRCh37-v4.2.1-stratifications.tsv"
